This notebook implements reclassification of gullies through supervised classification.

In [1]:
%matplotlib inline
import os
import datacube
import warnings
import numpy as np
import geopandas as gpd
import pandas as pd
import xarray as xr
import rioxarray
from rasterio.enums import Resampling
from odc.io.cgroups import get_cpu_quota
from odc.algo import xr_geomedian
from datacube.utils.cog import write_cog
from deafrica_tools.datahandling import load_ard
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.classification import collect_training_data, predict_xr
from deafrica_tools.spatial import xr_rasterize
from sklearn.ensemble import RandomForestClassifier
from random_sampling import random_sampling # adapted from function by Chad Burton: https://gist.github.com/cbur24/04760d645aa123a3b1817b07786e7d9f
from fast_glcm import fast_glcm_contrast # fast glcm metrics calculation, adapted from https://github.com/tzm030329/GLCM
from odc.algo import xr_reproject
import time
import matplotlib.pyplot as plt
from deafrica_tools.dask import create_local_dask_cluster

# get number of cpus
ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))

# parameters
crs='epsg:4326' # input crs: WGS84
output_crs='epsg:32735' # output crs: WGS84/UTM Zone 35S
class_name = 'LC_Class_I' # class label in integer format
n_samples=2000 # number of random samples to train classifier
fill_nan_value=-999 # value to replace nans in query results
measurements = ['blue','green','red','red_edge_1','red_edge_2','red_edge_3','nir_1','swir_1','swir_2'] # band measurements for data query
measurements_subset=['blue','green','red','nir_1'] # band measurements for texture calculation

# file paths and attributes
lesotho_tiles_shp='Data/Lesotho_boundaries_projected_epsg32735_tiles.shp' # Lesotho tiles shapefile
road_network_shp='Data/road_network.shp' # OSM road network data
classification2015_raster='Data/classification2015.tif' # land cover map of 2015
classification2021_raster='Results/Land_cover2021_postproc_step1_mosaic.tif' # land cover map of 2021
hand_raster='Data/hand_Lesotho.tif' # Hydrologically adjusted elevations, i.e. height above the nearest drainage (hand)
path_gully_signatures = "Results/training_data_gully.txt" # gully signature data

# import Lesotho tiles and get bounding box
lesotho_tiles=gpd.read_file(lesotho_tiles_shp).to_crs(crs) 
tile_bboxes=lesotho_tiles.bounds

# load land cover maps
landcover2021=rioxarray.open_rasterio(classification2021_raster).astype(np.uint8).squeeze() # import land cover map of 2021
landcover2015=rioxarray.open_rasterio(classification2015_raster).astype(np.uint8).squeeze() # import land cover map of 2015

# remapping class values of 2015 land cover map to be consistent with 2021
original_class=[11, 12, 21, 22, 23, 24, 31, 33, 41, 42, 43, 44, 51, 61, 62, 71, 72, 73, 74, 75, 13, 14, 25, 32, 34, 35, 36, 37, 52]
mapped_class=[1, 1, 2, 2, 3, 14, 4, 5, 6, 6, 7, 8, 9, 10, 11, 12, 12, 12, 15, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0]
landcover2015_mapped=landcover2015.copy()
crs_copy_2015=landcover2015.rio.crs
for i in range(len(original_class)):
    landcover2015_mapped=xr.where(landcover2015==original_class[i],mapped_class[i],landcover2015_mapped)
del landcover2015 # delete variable to release memory
if landcover2015_mapped.rio.crs is None: # reassign crs which was lost during last step of using xr.where
    landcover2015_mapped.rio.write_crs(crs_copy_2015,inplace=True)
if landcover2015_mapped.rio.crs!=output_crs: # reproject 2015 land cover map if needed
    landcover2015_mapped=landcover2015_mapped.rio.reproject(resolution=10, dst_crs=output_crs,resampling=Resampling.nearest)
    
# load external layers
road_network=gpd.read_file(road_network_shp).to_crs(output_crs) # import OSM road network data and reproject
road_network=road_network.loc[road_network['surface'].isin(['asphalt', 'paved', 'compacted', 'cobblestone', 
                                                             'concrete', 'metal', 'paving_stones', 
                                                             'paving_stones:30'])] # select road network by attributes
road_network.geometry=road_network.geometry.buffer(10) # buffer the road network by 10m
hand=xr.open_dataset(hand_raster,engine="rasterio").squeeze() # import hand layer

ncpus = 62
Lethoso bbox:  27.011232336601374 -30.67784748254426 29.4573649650311 -28.57059736718119


In [2]:
# read in or extract gully training signatures through randomly distributed sample points
if os.path.exists(path_gully_signatures): # if exists, read in 
    pd_gully_signatures= pd.read_csv(path_gully_signatures,delimiter=' ')
    column_names_gully=list(pd_gully_signatures.columns) # get feature names 
else: # otherwise extract gully signatures stratified random sample points
    landcover2015_mapped=landcover2015_mapped.where(landcover2015_mapped!=0,np.nan) # replace nodata class values 0 to nan so that it won't be used in analysis
    gdf_samples=random_sampling(landcover2015_mapped,n_samples,sampling='stratified_random',
                                manual_class_ratios=None,out_fname=None) # generate random samples
    gdf_samples['class']=gdf_samples['class'].astype(int) # convert data type to integer as required by collect_training_data
    if gdf_samples.crs is None: # assign crs if not exists
        gdf_samples=gdf_samples.set_crs(output_crs)
#     gdf_samples.to_file('Results/gully_classification_random_training_samples.geojson', driver="GeoJSON")

    # buffer training points so that neighbouring pixels are available for texture calculation
    # then extract texture feature values of original pixels
    gdf_samples_buffered=gdf_samples.copy() # initialise buffered geodataframe
    gdf_samples_buffered['geometry']=gdf_samples['geometry'].buffer(30) # buffering by 30 m
    
    # define query
    query = {
    'time': ('2021-01', '2021-02'), # NOTE: here we only uses Jan-Feb geomedian
    'measurements': measurements,
    'resolution': (-10, 10),
    'output_crs':output_crs,
    }
    # define a function to feature layers
    def feature_func_gully(query):
        #connect to the datacube
        dc = datacube.Datacube(app='feature_layers_gully')
        # query bands
        ds = load_ard(dc=dc,products=['s2_l2a'],
                      group_by='solar_day',
                      verbose=False,
                      **query)
        ds = calculate_indices(ds,index=['NDVI'],drop=False,collection='s2')
        # calculate geomedian
        ds=xr_geomedian(ds)
    #     ds=ds.resample(time='2MS').map(xr_geomedian).squeeze('time', drop=True)
    
        # Generate the GLCM contrast and correlation metrics for four bands with a window size of 3 pixels
        prop='contrast'
        for band in measurements_subset:
            var_name=band+'_'+prop
            if band=='nir_1':
                vmax=5000
            else:
                vmax=3000
            texture_array=fast_glcm_contrast(ds[band].to_numpy(),
                                             vmin=0,vmax=vmax,levels=8,
                                             ks=3,distance=1,angles=[0,45,90,135])
            texture_da=xr.DataArray(data=texture_array, name=var_name,
                                    coords={'y': ds.y.to_numpy(),'x': ds.x.to_numpy()},
                                    dims=['y','x'])
            texture_da.attrs=ds['blue'].attrs
            ds=xr.merge([ds,texture_da])
        return ds
    
    column_names_gully_buffered, training_data_gully_buffered = collect_training_data(gdf=gdf_samples_buffered,
                                                      dc_query=query,
    #                                                       ncpus=ncpus,
                                                      ncpus=10,
                                                      field='class',
                                                      zonal_stats=None,
                                                      feature_func=feature_func_gully,
                                                      return_coords=True) # collect training signatures
    pd_gully_signatures_buffered=pd.DataFrame(data=training_data_gully_buffered,columns=column_names_gully_buffered) # generate geopandas dataframe
    
    # identify signatures corresponding to original sampling points
    gpd_gully_signatures_buffered=gpd.GeoDataFrame(pd_gully_signatures_buffered, geometry=gpd.points_from_xy(pd_gully_signatures_buffered.x_coord, 
                                                                                      pd_gully_signatures_buffered.y_coord,
                                                                                      crs=output_crs)) # generate geopandas dataframe
    pd_gully_signatures=gdf_samples.drop(['class'],axis=1).sjoin_nearest(gpd_gully_signatures_buffered,how='left') # join sampling points with nearest coordinates
    pd_gully_signatures=pd_gully_signatures.drop_duplicates(subset='geometry', keep='first') # removing duplicated points
    column_names_gully=column_names_gully_buffered[:-2] # remove coordinate columns from features
    pd_gully_signatures=pd_gully_signatures[column_names_gully] # keep only features needed
    pd_gully_signatures.to_csv(path_gully_signatures, header=True, index=None, sep=' ') # save as txt file

training_data_gully=pd_gully_signatures.to_numpy()
print('Column names of extracted gully training data:\n',column_names_gully)
print('Extracted gully training data:\n',training_data_gully)
# train a random forest classifier
rf=RandomForestClassifier(n_estimators=100,max_samples=0.5,min_samples_leaf=1,bootstrap=True,n_jobs=1)
rf.fit(training_data_gully[:,1:],training_data_gully[:,0])
print('feature importance:\n',rf.feature_importances_)

Column names of extracted gully training data:
 ['class', 'blue', 'green', 'red', 'red_edge_1', 'nir_1', 'swir_1', 'swir_2', 'NDVI', 'blue_contrast', 'green_contrast', 'red_contrast', 'nir_1_contrast']
Extracted gully training data:
 [[1.00000000e+01 5.01775787e+02 7.15675171e+02 ... 4.00000000e+00
  1.50000000e+00 2.50000000e-01]
 [1.00000000e+01 4.72591980e+02 7.52156311e+02 ... 4.50000000e+00
  4.25000000e+00 3.00000000e+00]
 [1.00000000e+01 3.60964966e+02 5.71942932e+02 ... 2.50000000e+00
  6.50000000e+00 4.00000000e+00]
 ...
 [5.00000000e+00 2.91218323e+02 4.81856262e+02 ... 0.00000000e+00
  1.50000000e+00 5.50000000e+00]
 [4.00000000e+00 2.93633301e+02 5.65374756e+02 ... 7.50000000e-01
  2.50000000e-01 3.25000000e+00]
 [1.30000000e+01 7.48151672e+02 8.13986572e+02 ... 4.00000000e+00
  4.00000000e+00 4.25000000e+00]]
feature importance:
 [0.09046499 0.08879363 0.09717845 0.10106689 0.10405131 0.09299244
 0.09217528 0.09961853 0.05298282 0.05167288 0.05831925 0.07068352]


In [3]:
# # Set up a dask cluster
create_local_dask_cluster(n_workers=20)
# create_local_dask_cluster()
# for i in range(len(tile_bboxes)):
for i in range(0,11):
    x_min,y_min,x_max,y_max=tile_bboxes.iloc[i]
    print('Processing tile ',i,'with bbox of ',x_min,y_min,x_max,y_max)
    start_time = time.time()
    measurements = ['blue','green','red','red_edge_1','nir_1','swir_1','swir_2']
    query = {
    'x': (x_min,x_max),
    'y': (y_min,y_max),
    'time': ('2021-01', '2021-02'),
    'measurements': measurements,
    'resolution': (-10, 10),
    'crs':crs,
    'output_crs':output_crs,
    'dask_chunks' : {'x':1000, 'y':1000}
    }
    #connect to the datacube
    dc = datacube.Datacube(app='feature_layers_gully')
    ds = load_ard(dc=dc,products=['s2_l2a'],
                  group_by='solar_day',
                  verbose=False,
                  **query)
    ds_indices = calculate_indices(ds,index=['NDVI'],drop=False,collection='s2')
    # calculate geomedian
#     ds_geomedian=xr_geomedian(ds_indices).compute()
    ds_geomedian=xr_geomedian(ds_indices).compute()
    print(ds_geomedian)
    print("%s seconds spent on data query for one tile" % (time.time() - start_time))
    start_time = time.time()
    # calculate GLCM texture for four bands
    # Generate the contrast and correlation metrics with a window size of 3 pixels
    prop='contrast'
    for band in measurements_subset:
        print('calculating texture for band ',band)
        var_name=band+'_'+prop
        if band=='nir_1':
            vmax=5000
        else:
            vmax=3000
        texture_array=fast_glcm_contrast(ds_geomedian[band].to_numpy(),
                                         vmin=0,vmax=vmax,levels=8,
                                         ks=3,distance=1,angles=[0,45,90,135])
        texture_da=xr.DataArray(data=texture_array, name=var_name,
                                coords={'y': ds_geomedian.y.to_numpy(),'x': ds_geomedian.x.to_numpy()},
                                dims=['y','x'])
        texture_da.attrs=ds_geomedian['blue'].attrs
        ds_geomedian=xr.merge([ds_geomedian,texture_da])
    ds_geobox=ds_geomedian.geobox
    print('geobox for integrated NDVI dataset: ',ds_geobox)
    print('geomedian dataset for gully classification:\n',ds_geomedian)
    print("%s seconds spent on texture calculation for one tile" % (time.time() - start_time))
    
    df_all_data_with_texture=ds_geomedian.to_dataframe()
    np_all_data_with_texture=df_all_data_with_texture[column_names_gully[1:]].to_numpy()
    indices_nan=np.argwhere(np.isnan(np_all_data_with_texture).any(axis=1)) # find rows with nan
    print('number of pixels with nan: ',len(indices_nan))
    np_all_data_with_texture[np.isnan(np_all_data_with_texture)]=fill_nan_value # fill nan so that predcition can be used
    predicted_classes=rf.predict(np_all_data_with_texture)
    predicted_classes[indices_nan]=0 # assign those pixels with non-gully class
    print('predicted classes: \n',predicted_classes)
    # reshape cluster results to 2D array
    predicted_classes=np.reshape(predicted_classes,(ds_geomedian.dims['y'],ds_geomedian.dims['x']))
    # generate gully mask based on the predicted result
    np_gullies_mask=predicted_classes==15
    # reclassify as gullies where land cover was cropland or bare soil in 2015,
    # now classified as gullies, not overlapping road network and height above nearest drainge < 45m
    landcover2015_mapped_tile=xr_reproject(landcover2015_mapped.to_dataset(name='band_data'), ds_geobox, resampling="nearest") # clip to tile boundary
    np_landcover2015_mapped=landcover2015_mapped_tile.to_array().squeeze().to_numpy()
    road_network_mask=xr_rasterize(gdf=road_network,
                                      da=landcover2015_mapped_tile.to_array().squeeze(),
                                      transform=ds_geobox.transform,
                                      crs=output_crs)
    np_road_network_mask=road_network_mask.to_numpy()
    # load hand layer
    hand=xr_reproject(hand, ds_geobox, resampling="average")
    np_hand=hand.to_array().squeeze().to_numpy()
    landcover2021_tile=xr_reproject(landcover2021.to_dataset(name='band_data'), ds_geobox, resampling="nearest") # clip to tile boundary
    np_landcover2021=landcover2021_tile.to_array().squeeze().to_numpy()
    np_landcover2021_post=np_landcover2021.copy()
    np_landcover2021_post[((np_landcover2015_mapped==2)|(np_landcover2015_mapped==3)|(np_landcover2015_mapped==12))
                  &(np_hand<=45)&(np_road_network_mask==0)&(np_gullies_mask==1)]=15
    # carry over gullies from 2015 land cover map
    np_landcover2021_post[np_landcover2015_mapped==15]=15
    # convert back result back to DataArray
    landcover2021_tile_post=xr.DataArray(data=np_landcover2021_post,
                                         dims=['y','x'],
                                         coords={'y':landcover2021_tile.y.to_numpy(),
                                                 'x':landcover2021_tile.x.to_numpy()})
    landcover2021_tile_post.rio.write_crs(output_crs, inplace=True)
    # export as geotiff
    write_cog(landcover2021_tile_post, 'Results/Land_cover2021_postproc_step2_tile_'+str(i)+'.tif', overwrite=True)

Client Scheduler: tcp://127.0.0.1:43005 Dashboard: /user/whusggliuqx@gmail.com/proxy/8787/status,Cluster Workers: 20 Cores: 80 Memory: 512.40 GB


Processing tile  0 with bbox of  27.0111647418473 -29.48296693924553 27.526967799971306 -29.03066178308277


CPLReleaseMutex: Error = 1 (Operation not permitted)
CPLReleaseMutex: Error = 1 (Operation not permitted)
CPLReleaseMutex: Error = 1 (Operation not permitted)
CPLReleaseMutex: Error = 1 (Operation not permitted)


<xarray.Dataset>
Dimensions:     (y: 5024, x: 5024)
Coordinates:
  * y           (y) float64 6.789e+06 6.789e+06 ... 6.738e+06 6.738e+06
  * x           (x) float64 5.011e+05 5.011e+05 ... 5.513e+05 5.513e+05
Data variables:
    blue        (y, x) float32 455.5 450.9 472.6 455.7 ... 463.2 471.5 530.1
    green       (y, x) float32 768.1 762.5 798.6 766.0 ... 723.3 781.7 879.7
    red         (y, x) float32 888.5 863.9 941.4 890.0 ... 591.4 663.2 840.3
    red_edge_1  (y, x) float32 1.317e+03 1.319e+03 ... 1.228e+03 1.253e+03
    nir_1       (y, x) float32 2.073e+03 2.051e+03 ... 2.755e+03 2.736e+03
    swir_1      (y, x) float32 2.82e+03 2.825e+03 ... 2.047e+03 2.058e+03
    swir_2      (y, x) float32 1.946e+03 1.944e+03 ... 1.264e+03 1.255e+03
    NDVI        (y, x) float32 0.4007 0.4079 0.384 ... 0.6509 0.6033 0.525
1643.8275246620178 seconds spent on data query for one tile
calculating texture for band  blue
calculating texture for band  green
calculating texture for band  red
calcu

In [1]:
! gdal_merge.py -o Results/Land_cover2021_postproc_step2_mosaic.tif -co COMPRESS=Deflate -ot Byte Results/Land_cover2021_postproc_step2_tile_*.tif

0...10...20...30...40...50...60...70...80...90...100 - done.
